In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path
from datetime import datetime


In [2]:
#Redis Credentials
REDIS_HOST = 'redis-10596.c135.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 10596
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'iW4IWt7Jp1V6tdd6ZggZzwKRQDEp1nlE'

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


In [3]:
def is_valid_blt(blt):
        try:
            blt = int(blt)
            if blt >=0 and blt <=100:
                return True
            else:
                return False
        except ValueError:
            return False
    
def is_valid_plugged(plugged):
        try:
            plugged = int(plugged)
            if plugged ==0 or plugged ==1:
                return True
            else:
                return False
        except ValueError:
            return False

def is_valid_date_format(date_string, date_format="%Y-%m-%d"):
    try:
        parsed_date = datetime.strptime(date_string, date_format)
        return True
    except ValueError:
        return False

In [4]:
#endpoint /devices
class DevicesList(object):
    exposed = True

    def GET(self, *path, **query):
        '''Retrieve the list of MAC address of the monitored devices'''
        mac_addresses = []      #list to store all the monitored device
        keys = redis_client.keys('*:power')   #mac_address:power considering all the possible devices 

        blt = query.get('blt', None)   #battery threshold
        plugged = query.get('plugged', None)  #plugged flag

        if blt is not None: #check if it is a valid value
            valid_blt = is_valid_blt(blt)
            if valid_blt == False:
                raise cherrypy.HTTPError(400, 'Bad Request: invalid battery threshold value.')
            else:
                blt = int(blt)

        if plugged is not None:  #check if it is a valid value
            valid_plugged = is_valid_plugged(plugged)
            if valid_plugged == False:
                raise cherrypy.HTTPError(400, 'Bad Request: invalid plugged value.')
            else:
                plugged = int(plugged)
                 
        for key in keys:
            key = key.decode()
            mac_address = key.removesuffix(':power')
            last_plugged_value = int(redis_client.execute_command("TS.RANGE", f'{mac_address}:power', "-", "+", "COUNT", "1")[0][1])
            last_battery_value = int(redis_client.execute_command("TS.RANGE", f'{mac_address}:battery', "-", "+", "COUNT", "1")[0][1])

            if blt is not None and plugged is not None:
                    if last_plugged_value == plugged and last_battery_value <= blt:
                        mac_addresses.append(mac_address)   
            elif blt is not None:
                if last_battery_value <= blt:
                    mac_addresses.append(mac_address)
            elif plugged is not None:
                if last_plugged_value == plugged:
                    mac_addresses.append(mac_address)
            else:
                mac_addresses.append(mac_address)
            
        response = json.dumps({'mac_addresses':mac_addresses})

        cherrypy.response.status = '200 OK: Everything worked as expected.'
        

        return response

In [5]:
class DeviceDetail(object):
    exposed = True

    def GET(self, *path, **query):
        #check if the MAC address has been specified
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')
        
        mac_address = path[0]
        
        start_date = query.get('start_date', None)
        end_date = query.get('end_date', None)
        if start_date is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start date.')
        if end_date is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end date.')
        
        #check the date format --- TO DO
        valid_start_date = is_valid_date_format(start_date)
        valid_end_date = is_valid_date_format(end_date)
        if valid_start_date == False:
            raise cherrypy.HTTPError(400, 'Bad Request: wrong format for start date.')
        if valid_end_date == False:
            raise cherrypy.HTTPError(400, 'Bad Request: wrong format for end date.')

        
        start_timestamp = int(datetime.fromisoformat(start_date).timestamp() * 1000)
        end_timestamp = int(datetime.fromisoformat(end_date).timestamp() * 1000)

        if start_timestamp >= end_timestamp:
            raise cherrypy.HTTPError(400, 'Bad Request: end date smaller or equal than start date.')

        if not redis_client.exists(f'{mac_address}:power'):
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')
        
        battery = redis_client.execute_command("TS.RANGE", f'{mac_address}:battery', start_timestamp, end_timestamp)
        power = redis_client.execute_command("TS.RANGE", f'{mac_address}:power', start_timestamp, end_timestamp)
 

        timestamps = []
        battery_values = []
        power_plugged_values = []

        if battery:
            for timestamp, battery_value in battery:
                timestamps.append(int(timestamp))
                battery_values.append(int(battery_value))
        
        if power:
            for timestamp, power_value in power:
                power_plugged_values.append(int(power_value))

    
        response = json.dumps({'mac_address': mac_address, 'timestamps':timestamps, 'battery_levels': battery_values, 'power_plugged': power_plugged_values})

        cherrypy.response.status = '200 OK: Everything worked as expected.'

        return response
    
    def DELETE(self, *path, **query):
        #check if the MAC address has been specified
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')
        mac_address = path[0]

        if not redis_client.exists(f'{mac_address}:power'):
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')

        redis_client.delete(f'{mac_address}:power')
        redis_client.delete(f'{mac_address}:battery')

        cherrypy.response.status = '200 OK: Everything worked as expected.'

        return

In [ ]:
if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(DevicesList(), '/devices', conf)
    cherrypy.tree.mount(DeviceDetail(), '/device', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0ec8e575-19c8-4894-9160-01c0b36ec399' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>